In [13]:
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import re
import cv2
import json
from pprint import pprint
import random
from sklearn.model_selection import StratifiedKFold
from monkey.data.data_utils import (
    open_json_file,
    write_json_file,
    extract_id,
)

In [2]:
wsi_dir = "/home/u1910100/Downloads/Monkey/images/pas-cpg"
wsi_names = os.listdir(wsi_dir)
wsi_names.sort()
# pprint(wsi_names)

split_labels = []
for name in wsi_names:
    centre = name[0]
    if centre == "A":
        split_labels.append(0)
    elif centre == "B":
        split_labels.append(1)
    elif centre == "C":
        split_labels.append(2)
    else:
        split_labels.append(3)

# pprint(split_labels)

In [3]:
skf = StratifiedKFold(n_splits=4, random_state=None, shuffle=False)
skf.get_n_splits(wsi_names, split_labels)
for i, (train_index, test_index) in enumerate(
    skf.split(wsi_names, split_labels)
):
    pprint(f"Fold {i}:")
    pprint(f"Train: index={train_index}")
    pprint(f"Test:  index={test_index}")

'Fold 0:'
('Train: index=[ 7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 30 31 '
 '32 33 34\n'
 ' 35 36 37 38 39 40 41 42 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 68\n'
 ' 69 70 71 72 73 74 75 76 77 78 79 80]')
'Test:  index=[ 0  1  2  3  4  5  6 26 27 28 29 43 44 45 46 47 63 64 65 66 67]'
'Fold 1:'
('Train: index=[ 0  1  2  3  4  5  6 14 15 16 17 18 19 20 21 22 23 24 25 26 27 '
 '28 29 34\n'
 ' 35 36 37 38 39 40 41 42 43 44 45 46 47 53 54 55 56 57 58 59 60 61 62 63\n'
 ' 64 65 66 67 72 73 74 75 76 77 78 79 80]')
'Test:  index=[ 7  8  9 10 11 12 13 30 31 32 33 48 49 50 51 52 68 69 70 71]'
'Fold 2:'
('Train: index=[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 20 21 22 23 24 25 26 '
 '27 28 29\n'
 ' 30 31 32 33 39 40 41 42 43 44 45 46 47 48 49 50 51 52 58 59 60 61 62 63\n'
 ' 64 65 66 67 68 69 70 71 76 77 78 79 80]')
'Test:  index=[14 15 16 17 18 19 34 35 36 37 38 53 54 55 56 57 72 73 74 75]'
'Fold 3:'
('Train: index=[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19

In [10]:
# Save WSI level split info
wsi_split_info = {}
for i, (train_index, test_index) in enumerate(
    skf.split(wsi_names, split_labels)
):
    # pprint(f"Fold {i}:")
    train_files = []
    test_files = []
    train_files = [wsi_names[i] for i in train_index]
    test_files = [wsi_names[i] for i in test_index]
    # pprint(f"Train files:={train_files}")
    # pprint(f"Test files={test_files}")

    wsi_split_info[f"Fold_{i+1}"] = {
        "train_files": train_files,
        "test_files": test_files,
    }

pprint(wsi_split_info)

save_path = (
    "/home/u1910100/Documents/Monkey/patches_256/wsi_level_split.json"
)
write_json_file(save_path, wsi_split_info)

{'Fold_1': {'test_files': ['A_P000001_PAS_CPG.tif',
                           'A_P000002_PAS_CPG.tif',
                           'A_P000003_PAS_CPG.tif',
                           'A_P000004_PAS_CPG.tif',
                           'A_P000005_PAS_CPG.tif',
                           'A_P000006_PAS_CPG.tif',
                           'A_P000007_PAS_CPG.tif',
                           'B_P000001_PAS_CPG.tif',
                           'B_P000002_PAS_CPG.tif',
                           'B_P000003_PAS_CPG.tif',
                           'B_P000004_PAS_CPG.tif',
                           'C_P000021_PAS_CPG.tif',
                           'C_P000022_PAS_CPG.tif',
                           'C_P000023_PAS_CPG.tif',
                           'C_P000024_PAS_CPG.tif',
                           'C_P000025_PAS_CPG.tif',
                           'D_P000001_PAS_CPG.tif',
                           'D_P000002_PAS_CPG.tif',
                           'D_P000003_PAS_CPG.tif',
            

In [33]:
# Create patch level splits from wsi level splits

patches_stats_path = (
    "/home/u1910100/Documents/Monkey/patches_256/patch_stats.json"
)

patch_stats = open_json_file(patches_stats_path)

# pprint(patch_stats)

patch_names = list(patch_stats.keys())
patch_names.sort()
# pprint(patch_names)

wsi_name_patch_dict = {}

for name in patch_names:
    wsi_name = extract_id(name)
    wsi_name_patch_dict[name] = wsi_name

# pprint(wsi_name_patch_dict)
#     patch_ids.append(id)

# pprint(patch_ids)

patch_split_info = {}

for fold_data in wsi_split_info.items():
    fold_number = fold_data[0]
    train_wsis = fold_data[1]["train_files"]
    test_wsis = fold_data[1]["test_files"]

    train_wsi_ids = [extract_id(name) for name in train_wsis]
    test_wsi_ids = [extract_id(name) for name in test_wsis]

    # pprint(test_wsi_ids)
    train_patches = []
    test_patches = []

    for patch_item in wsi_name_patch_dict.items():
        patch_name = patch_item[0]
        wsi_id = patch_item[1]
        if wsi_id in test_wsi_ids:
            test_patches.append(patch_name)
        else:
            train_patches.append(patch_name)

    patch_split_info[fold_number] = {
        "train_files": train_patches,
        "test_files": test_patches,
    }

# pprint(patch_split_info)
save_path = "/home/u1910100/Documents/Monkey/patches_256/patch_level_split.json"
write_json_file(save_path, patch_split_info)